In [1]:
# Cell 1: install minimal libs + load zero-shot model
!pip install -q transformers pdfplumber python-docx gradio scikit-learn

import os, re, shutil, math
import torch
from transformers import pipeline

os.makedirs("uploads", exist_ok=True)
os.makedirs("converted", exist_ok=True)
os.makedirs("storage", exist_ok=True)

device = 0 if torch.cuda.is_available() else -1
print("Using device:", "cuda:0" if device==0 else "cpu")

print("Loading zero-shot model (this may take 1-2 minutes)...")
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)
print("Model loaded.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 52.7 MB/s eta 0:00:00
Using device: cuda:0
Loading zero-shot model (this may take 1-2 minutes)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Model loaded.


In [2]:
# Cell 2: improved SUBJECTS and robust multi-chunk zero-shot classifier
import pdfplumber, docx
from collections import Counter
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# ---------- Your subject list (exact) ----------
SUBJECTS = [
    "AWS",
    "Internet Web Programming",
    "Theory of Computation & Compiler Design",
    "Mathematics",
    "Indian Constitution",
    "Universal Human Values",
    "Java",
    "C++",
    "Python",
    "Data Structures & Algorithms",
    "Computer Networks",
    "Electrical Systems & Circuits",
    "Digital Logic Gates & Circuits",
    "Mechanics",
    "Bioinformatics",
    "Miscellaneous"
]

# Create storage folders
for s in SUBJECTS:
    folder = os.path.join("storage", s.replace(" ", "_"))
    os.makedirs(folder, exist_ok=True)

# Add short descriptive phrases for each subject to help NLI (keeps label-to-subject mapping stable)
LABEL_PROMPTS = {
    "AWS": "AWS (Amazon Web Services, cloud infrastructure, EC2, S3, cloud computing)",
    "Internet Web Programming": "Internet Web Programming (HTML, CSS, JavaScript, web development, HTTP)",
    "Theory of Computation & Compiler Design": "Theory of Computation and Compiler Design (automata, parsing, compilers)",
    "Mathematics": "Mathematics (algebra, calculus, proofs)",
    "Indian Constitution": "Indian Constitution (constitution law, governance)",
    "Universal Human Values": "Universal Human Values (ethics, values, human behavior)",
    "Java": "Java programming language",
    "C++": "C++ programming language",
    "Python": "Python programming language",
    "Data Structures & Algorithms": "Data Structures and Algorithms (trees, graphs, sorting)",
    "Computer Networks": "Computer Networks (TCP/IP, routing, networking)",
    "Electrical Systems & Circuits": "Electrical Systems and Circuits (circuits, voltage, current)",
    "Digital Logic Gates & Circuits": "Digital Logic (gates, flip-flops, combinational/sequential circuits)",
    "Mechanics": "Mechanics (physics: motion, forces, kinematics)",
    "Bioinformatics": "Bioinformatics (genomics, sequence analysis)",
    "Miscellaneous": "Miscellaneous (other or uncategorized educational content)"
}

# Build list in same order as SUBJECTS (so mapping is stable)
label_prompts_list = [LABEL_PROMPTS[s] for s in SUBJECTS]

# ---------------- text extraction ----------------
def extract_text(path):
    path = str(path)
    ext = path.lower().rsplit(".",1)[-1]
    text = ""
    try:
        if ext == "pdf":
            with pdfplumber.open(path) as pdf:
                pages = [p.extract_text() or "" for p in pdf.pages]
            text = "\n".join(pages)
        elif ext in ("docx", "doc"):
            doc = docx.Document(path)
            text = "\n".join([p.text for p in doc.paragraphs])
        elif ext == "txt":
            with open(path, "r", encoding="utf-8", errors="ignore") as f:
                text = f.read()
    except Exception as e:
        print("Extract error:", e)
        text = ""
    return text.strip()

# ---------------- chunking ----------------
def chunk_text(text, max_words=250):
    words = text.split()
    if len(words) <= max_words:
        return [text]
    chunks = []
    cur = []
    for w in words:
        cur.append(w)
        if len(cur) >= max_words:
            chunks.append(" ".join(cur))
            cur = []
    if cur:
        chunks.append(" ".join(cur))
    return chunks

# ---------------- simple keyword extraction (frequency-based) ----------------
def top_keywords(text, topk=8):
    words = re.findall(r"[A-Za-z]{2,}", text.lower())
    words = [w for w in words if w not in ENGLISH_STOP_WORDS]
    counts = Counter(words)
    return [w for w,_ in counts.most_common(topk)]

# ---------------- robust zero-shot classification ----------------
def classify_document(path, threshold=0.28, max_chunks=6, debug=False):
    """
    Returns: (predicted_subject, score, detail_dict)
    - averages scores across up to max_chunks of the document (head, mid, tail)
    - uses filename signals (e.g., 'aws' in name) to boost label
    - threshold decides minimum confidence to avoid Miscellaneous
    """
    fname = os.path.basename(path)
    fname_low = fname.lower()
    text = extract_text(path)
    if not text:
        return "Miscellaneous", 0.0, {"reason":"no_text"}

    # quick filename check (strong signal)
    if re.search(r"\baws\b", fname_low) or "amazon web services" in text.lower():
        return "AWS", 0.99, {"reason":"filename_or_text_contains_aws"}

    # prepare chunks: head, middle, tail (and additional chunks up to max_chunks)
    words = text.split()
    if len(words) < 250:
        chunks = [text]
    else:
        chunks = []
        N = len(words)
        # head
        chunks.append(" ".join(words[:250]))
        # middle
        mid_start = max(0, N//2 - 125)
        chunks.append(" ".join(words[mid_start: mid_start+250]))
        # tail
        chunks.append(" ".join(words[-250:]))
        # if still want more, get sequential chunks
        ptr = 250
        while len(chunks) < max_chunks and ptr+250 < N:
            chunks.append(" ".join(words[ptr:ptr+250]))
            ptr += 250

    # also augment the first chunk with top keywords (helps short docs)
    kws = top_keywords(text, topk=6)
    if kws:
        chunks[0] = chunks[0] + " " + " ".join(kws)

    # run classifier on each chunk, collect scores per label-prompt
    import numpy as np
    accum_scores = np.zeros(len(label_prompts_list), dtype=float)
    count = 0
    for ch in chunks:
        out = classifier(ch, label_prompts_list, hypothesis_template="This document is about {}.", multi_label=True)
        # out["labels"] is ordered by descending score; but we need the full vector of scores
        # pipeline returns 'labels' and 'scores' arrays aligned
        labels = out["labels"]
        scores = out["scores"]
        # map scores to our label_prompts_list ordering
        score_map = {lab: sc for lab, sc in zip(labels, scores)}
        # some labels might be missing if multi_label False; but with multi_label True we get all candidates
        this_scores = [score_map.get(lp, 0.0) for lp in label_prompts_list]
        accum_scores += np.array(this_scores)
        count += 1

    avg_scores = accum_scores / max(1, count)
    # find best index
    best_idx = int(np.argmax(avg_scores))
    best_score = float(avg_scores[best_idx])
    predicted_prompt = label_prompts_list[best_idx]
    predicted_subject = SUBJECTS[best_idx]

    # if score below threshold -> Miscellaneous
    if best_score < threshold:
        return "Miscellaneous", best_score, {"avg_scores": dict(zip(SUBJECTS, avg_scores.tolist())), "reason":"low_confidence"}

    # otherwise save converted text and return
    # save converted text
    base = os.path.basename(path)
    conv_path = os.path.join("converted", base + ".txt")
    with open(conv_path, "w", encoding="utf-8") as f:
        f.write(text)

    # copy into storage folder
    dest_folder = os.path.join("storage", predicted_subject.replace(" ", "_"))
    os.makedirs(dest_folder, exist_ok=True)
    shutil.copy(conv_path, os.path.join(dest_folder, base + ".txt"))

    details = {
        "best_prompt": predicted_prompt,
        "best_score": best_score,
        "avg_scores": dict(zip(SUBJECTS, [float(x) for x in avg_scores])),
        "used_chunks": len(chunks)
    }
    if debug:
        print("DEBUG details:", details)
    return predicted_subject, best_score, details


In [4]:
# Cell 3: Gradio UI to upload files and run the classifier
import gradio as gr
import os

def run_classification(filepaths):
    rows = []
    for path in filepaths:
        # path is a local path (gradio gives filepath when type="filepath")
        pred, score, info = classify_document(path, threshold=0.28, debug=False)
        msg = f"score={score:.3f}"
        rows.append([os.path.basename(path), pred, msg])
    return rows

with gr.Blocks() as demo:
    gr.Markdown("## 📂 Zero-Shot Academic File Classifier (multi-chunk + filename signal)")
    gr.Markdown("Upload one or more PDF/DOCX/TXT files. Uses multi-chunk zero-shot to decide best subject. Low-confidence -> Miscellaneous.")
    file_input = gr.File(file_types=[".pdf", ".docx", ".txt"], type="filepath", label="Upload Files", file_count="multiple")
    classify_btn = gr.Button("Classify & Store")
    output_table = gr.Dataframe(headers=["File", "Predicted Subject", "Message"], row_count=(0,"dynamic"))
    classify_btn.click(fn=run_classification, inputs=file_input, outputs=output_table)

# Launch with share=True for a public URL in Colab
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1bc0bea8581da6f06e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
